# Mustaeen Ahmed: Assignment 9

In [ ]:
import pygame
import moderngl
from pyglm import glm
from loadModelUsingAssimp_V2 import create3DAssimpObject

In [ ]:
pygame.init()

WIDTH = 840
HEIGHT = 480

clock = pygame.time.Clock()
running = True

pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 3)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_FORWARD_COMPATIBLE_FLAG, True)

In [ ]:
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.set_mode((WIDTH, HEIGHT), flags=pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)
pygame.display.set_caption(title="Assignment 09: Mustaeen Ahmed")
gl = moderngl.get_context()
gl.info['GL_VERSION']

In [ ]:
assimp_object = create3DAssimpObject("chair_table_class/scene.gltf", verbose=False, textureFlag=True, normalFlag=True)

In [ ]:
bound = assimp_object.bound

displacement_vector = 2 * bound.radius * glm.vec3(0, 0, 1)
light_displacement_vector = 2 * bound.radius * glm.rotate(glm.vec3(0, 1, 0), glm.radians(45), glm.vec3(1, 0, 0))

target_point = glm.vec3(bound.center)
up_vector = glm.vec3(0, 1, 0)

fov_radian = glm.radians(45)
aspect = WIDTH / HEIGHT
near = bound.radius
far = 3 * bound.radius
perspective_matrix = glm.perspective(fov_radian, aspect, near, far)

In [ ]:
vertex_shader_code = '''
    #version 460 core
    
    layout (location=0) in vec3 position;
    layout (location=1) in vec2 uv;
    layout (location=2) in vec3 normal;

    uniform mat4 model, view, perspective;
    uniform mat3 normalMatrix;

    out vec2 f_uv;
    out vec3 f_normal;
    out vec3 f_position;

    void main() {
        f_uv = uv;
        vec4 P = (model * vec4(position, 1));
        f_position = P.xyz;

        gl_Position = perspective * view * P;
        mat3 normalMatrix = mat3(transpose(inverse(model)));
        f_normal = normalize(normalMatrix * normal);
    }
'''

fragment_shader_code = '''
    #version 460 core

    in vec2 f_uv;
    in vec3 f_normal;
    in vec3 f_position;

    uniform sampler2D map;
    uniform vec3 light;

    uniform float shininess;
    uniform vec3 eye_position;
    uniform vec3 k_diffuse;

    out vec4 out_color;

    vec3 computeColor() {
        vec3 L = normalize(light.xyz);
        vec3 materialColor = texture(map, f_uv).rgb;
        vec3 N = normalize(f_normal);

        float NdotL = dot(N, L);
        vec3 color = vec3(0.);

        if (NdotL > 0.) {
            vec3 ambientColor = materialColor * 0.1;
            vec3 diffuselyReflectedColor = materialColor * NdotL;

            vec3 V = eye_position - f_position;
            vec3 H = normalize(L + V);
            vec3 specularlyReflectedColor = vec3(1.) * pow(dot(N, H), shininess);
            color = ambientColor + k_diffuse * diffuselyReflectedColor + specularlyReflectedColor;
        }
        return color;
    }

    void main() {
        out_color = vec4(computeColor(), 1);
    }
'''

In [ ]:
program = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)
f_mat = "3f 3f 2f"
variables = ["position", "normal", "uv"]
renderables = assimp_object.getRenderables(gl, program, f_mat, variables)

samplers = assimp_object.getSamplers(gl)

In [ ]:
scene = assimp_object.scene
meshes = scene.meshes
materials = scene.materials

def recursive_render(node, M):
    if node is None: 
        print("Empty node")
        return
    nodeTransform = glm.transpose(glm.mat4(node.transformation))
    currentTransform = M * nodeTransform
    if node.num_meshes > 0:
        for index in node.mesh_indices:
            samplers[index].use(0)
            program["map"] = 0
            program["model"].write(currentTransform)

            mesh = meshes[index]
            material = materials[mesh.material_index]
            program["shininess"] = material["SHININESS"]

            k_diffuse = glm.vec4(material["COLOR_DIFFUSE"]).rgb
            program["k_diffuse"].write(k_diffuse)

            renderables[index].render()

    for node in node.children:
        recursive_render(node, currentTransform)

def render():
    recursive_render(scene.root_node, glm.mat4(1))

In [ ]:
alpha = 0
pause = True
gl.enable(gl.DEPTH_TEST)
light_angle = 0

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif (event.type == pygame.KEYDOWN):
            if event.key == 27:
                running = False
            elif event.key == pygame.K_p:
                pause = not pause
            elif event.key == pygame.K_LEFT:
                light_angle -= 5
            elif event.key == pygame.K_RIGHT:
                light_angle += 5
        elif (event.type == pygame.WINDOWRESIZED):
            width = event.x
            height = event.y
            perspective_matrix = glm.perspective(fov_radian, width / height, near, far)

    new_displacement_vector = glm.rotate(displacement_vector, glm.radians(alpha), glm.vec3(0, 1, 0))
    new_light_displacement_vector = glm.rotate(light_displacement_vector, glm.radians(light_angle), glm.vec3(0, 1, 0))
    eye_point = target_point + new_displacement_vector
    viewMatrix = glm.lookAt(eye_point, target_point, up_vector)

    gl.clear(0.5, 0.5, 0.5)

    program['view'].write(viewMatrix)
    program['perspective'].write(perspective_matrix)
    program['light'].write(new_light_displacement_vector)
    program["eye_position"].write(eye_point)

    render()

    pygame.display.flip()
    clock.tick(60)
    if not pause:
        alpha = alpha + 1
        if alpha > 360:
            alpha = 0

pygame.display.quit()